## 1. Load Data

In [1]:
#Establish working directory
#os.getcwd()
#Mount Google Drive to Colab Enviorment
import os
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/capstone/data/
!pwd
#!ls
print('Working Directory')
print(os.getcwd())
work_dir = "/content/gdrive/My Drive/capstone/data/"

Mounted at /content/gdrive
/content/gdrive/My Drive/capstone/data
/content/gdrive/My Drive/capstone/data
Working Directory
/content/gdrive/My Drive/capstone/data


In [4]:
import pandas as pd
import numpy as np
from lifelines.utils import datetimes_to_durations
from lifelines import KaplanMeierFitter
from scipy.stats import iqr
from math import sqrt

In [5]:
# Read Data
sale1 = pd.read_excel(work_dir+"/Sales Data.xlsx")
sale2 = pd.read_excel(work_dir+"/Sales Data 2.xlsx")
sales = pd.concat([sale1, sale2])

In [ ]:
# ## AccountNumber(in account) = CustomerNo(in sales)
# ## FamilyCode_FNT__c (in account) = Family_Name__c (in industry)
# family = pd.read_excel(work_dir+'/Family Data.xlsx')
# account = pd.read_excel(work_dir+'/Account Data.xlsx')

### 1.1 Sales Data

In [6]:
sales.isnull().sum()
# Fill NAs
sales[['Item']] = sales[['Item']].fillna('Unknown')
# Negative value in Quantity Ordered and Line Item Amount represents return, so remove those
sales = sales[sales['Quantity Ordered'] > 0]

In [7]:
sales.shape

(1475087, 13)

In [8]:
sales.head()

,Customer No,Item,Quantity Ordered,Line Item Amount,Posting Date,Ship-to State,Ship-to ZipCode,Department Dim,Department Dim Description,Product Group Code,Product Group Description,Order Method,Location Code
2,0000919,DGISBNESS,1,799.0,2018-04-01,PA,15238,507,Software,NaN,NaN,LMS-RENEW,10
3,0000919,DGIS49CFR,1,0.0,2018-04-01,PA,15238,507,Software,NaN,NaN,LMS-RENEW,10
4,0000919,DGISINTLAIR,1,0.0,2018-04-01,PA,15238,507,Software,NaN,NaN,LMS-RENEW,10
5,0000919,DGISMARITIME,1,0.0,2018-04-01,PA,15238,507,Software,NaN,NaN,LMS-RENEW,10
7,0000919,DGISBNESS,1,799.0,2018-04-01,PA,15238,507,Software,NaN,NaN,LMS-RENEW,10


In [9]:
sales.isnull().sum()

Customer No                        0
Item                               0
Quantity Ordered                   0
Line Item Amount                   0
Posting Date                       0
Ship-to State                   3179
Ship-to ZipCode                  623
Department Dim                     0
Department Dim Description         0
Product Group Code            455325
Product Group Description     455394
Order Method                   13240
Location Code                   6042
dtype: int64

### 1.2 Family Data

In [ ]:
# family.shape

In [ ]:
# family.head()

In [ ]:
# # Check missing values of family code in family data
# family.isnull().sum()

### 1.3 Account Data

In [ ]:
# # Check missing values of FamilyCode & AccountNumber in account data
# account.isnull().sum()
# account = account.dropna(subset = ['AccountNumber', 'FamilyCode_FNT__c'])
# ## convert float to int
# account['FamilyCode_FNT__c'] = account['FamilyCode_FNT__c'].astype(int)

In [ ]:
# account.shape

In [ ]:
# account.head()

### 1.4 Combine Account & Family Data

In [ ]:
# # Merge account data and family data by FamilyCode_FNT__c (in account) = Family_Name__c (in family)
# ## changing column name 'FamilyCode_FNT__c' to 'Family_Name__c' & 'AccountNumber' to 'CustomerNo'
# accountdf = account.rename(columns={'FamilyCode_FNT__c': 'Family_Name__c', 'AccountNumber':'Customer No'})
# account_family = pd.merge(accountdf, family, on = 'Family_Name__c', how = 'outer', indicator = True)
# print('there are {} famliy code in BOTH dataframe'.format(account_family._merge.value_counts()[0]))
# print('there are {} famliy code ONLY in famliy dataset'.format(account_family._merge.value_counts()[1]))
# print('there are {} famliy code ONLY in account dataset'.format(account_family._merge.value_counts()[2]))

In [ ]:
# industry = account_family[account_family['_merge'] == 'both']
# industry.shape

In [ ]:
# industry.head()

### 1.5 Merge Account_Family & Sales Data

In [ ]:
# # Merge industry data and sales data by AccountNumber(industry) = CustomerNo(in sales)
# industry_sales = pd.merge(sales, industry[['Customer No', 'Family_Name__c', 'Family_Industry_Description_New__c']], on = 'Customer No', how = 'inner')
# industry_sales.shape

In [ ]:
# industry_sales.head()

In [ ]:
# industry_sales.isnull().sum()

### 1.6 Get useful columns, calculate days from 1st purchased to nth purchased, and days from purchased to a certain date

In [ ]:
# myData = industry_sales[['Customer No', 'Quantity Ordered', 'Line Item Amount', 
#                          'Posting Date', 'Department Dim', 'Department Dim Description',
#                          'Family_Name__c', 'Family_Industry_Description_New__c']]
# myData.shape

In [10]:
myData = sales[['Customer No', 'Quantity Ordered', 'Line Item Amount',
                'Posting Date', 'Department Dim Description', 'Product Group Description']]
myData.shape

(1475087, 6)

In [11]:
# Find first purchase date for each customer
firstPurchased = myData.groupby('Customer No')['Posting Date'].min().reset_index()
firstPurchased.columns = ['Customer No', 'First Purchase Date']
myData = pd.merge(myData, firstPurchased, on='Customer No')
myData['Days from 1st Purchase'], myData['Have not Purchased since 1st Purchase'] = datetimes_to_durations(myData['First Purchase Date'], myData['Posting Date'])
# # Find days of each purchase date to Today
# import datetime
# today = datetime.datetime(2022, 5, 1)
# myData['Days from Purchased to Now'] = today - myData['Posting Date']
# myData['Days from Purchased to Now'] = (myData['Days from Purchased to Now'] / np.timedelta64(1,'D')).astype(float)
# myData = myData.drop(columns=['Have not Purchased since 1st Purchase'])
# myData.shape
myData = myData.drop(columns = 'Have not Purchased since 1st Purchase')

In [12]:
myData.head()

,Customer No,Quantity Ordered,Line Item Amount,Posting Date,Department Dim Description,Product Group Description,First Purchase Date,Days from 1st Purchase
0,0000919,1,799.0,2018-04-01,Software,NaN,2016-01-11,811.0
1,0000919,1,0.0,2018-04-01,Software,NaN,2016-01-11,811.0
2,0000919,1,0.0,2018-04-01,Software,NaN,2016-01-11,811.0
3,0000919,1,0.0,2018-04-01,Software,NaN,2016-01-11,811.0
4,0000919,1,799.0,2018-04-01,Software,NaN,2016-01-11,811.0


In [13]:
myData['Customer No'].value_counts()

0409806    49869
3013961    34615
1623167    31348
1458331    27744
3002523    26177
           ...  
7040729        1
7014503        1
3028186        1
7040763        1
3036538        1
Name: Customer No, Length: 95540, dtype: int64

There are 95,540 identical customers in total

## 2. Group by Department

In [14]:
# Seperate into different Department
departmentName = myData['Department Dim Description'].unique()
departmentName

array(['Software', 'Packaging', 'Labels', 'Contra Dimension', 'Books',
       'Placards', 'Personal Compliance', 'Spacemaster',
       'Imprinted Placards', 'Facility Management', 'Amazon Labels',
       'None', 'Amazon Packaging', 'Marketing', 'Manufacturing',
       'Raw Material Paper', 'Recall Ready', 'Amerex Extinguishers',
       'Open', 'Packaging  and Warehouse', 'General Office',
       'LSI- Consulting', 'LSI- E-Learning', 'Fed-Ex US Development'],
      dtype=object)

In [15]:
myData['Department Dim Description'].value_counts()

Contra Dimension            425016
Labels                      333471
Placards                    257418
Books                        93409
Spacemaster                  81837
Packaging                    81344
Facility Management          62802
Personal Compliance          46106
Imprinted Placards           33351
Software                     31815
Amazon Labels                13007
Amazon Packaging              8297
LSI- Consulting               4577
Recall Ready                  1170
Marketing                      959
None                           422
Amerex Extinguishers            31
Fed-Ex US Development           23
LSI- E-Learning                 14
Packaging  and Warehouse         6
General Office                   5
Open                             4
Raw Material Paper               2
Manufacturing                    1
Name: Department Dim Description, dtype: int64

### 2.1 Example - look into Label Department

In [16]:
# Look at Labels only
Labels = myData[myData['Department Dim Description'] == 'Labels'].reset_index(drop=True)
Labels['Customer No'].value_counts()

0409806    22218
3041322    15412
1458331    10512
0267242     6433
3013961     6164
           ...  
3068618        1
7051959        1
7048058        1
7051940        1
3035860        1
Name: Customer No, Length: 44765, dtype: int64

In [ ]:
# # Visualize customers’ journey from first purchase to later purchases
# kmf = KaplanMeierFitter(label="customers")
# kmf.fit(myLabels['Days from 1st Purchase'], myLabels['Have not Purchased since 1st Purchase'])
# kmf.plot(figsize=(8,8), loc = slice(0., 365*2))

In [17]:
Labels

,Customer No,Quantity Ordered,Line Item Amount,Posting Date,Department Dim Description,Product Group Description,First Purchase Date,Days from 1st Purchase
0,0000919,10,197.30,2018-04-03,Labels,Forms,2016-01-11,813.0
1,0000919,5,104.25,2018-04-03,Labels,Forms,2016-01-11,813.0
2,0000919,10,616.00,2018-06-01,Labels,DOT Labels,2016-01-11,872.0
3,0000919,6,114.00,2016-01-12,Labels,DOT Labels,2016-01-11,1.0
4,0000919,24,312.00,2016-01-12,Labels,Air Labels,2016-01-11,1.0
...,...,...,...,...,...,...,...,...
333466,7037043,5,227.85,2021-01-14,Labels,Forms,2021-01-14,0.0
333467,7037050,1,62.20,2021-01-14,Labels,Air Labels,2021-01-14,0.0
333468,7037056,1,52.99,2021-01-14,Labels,Forms,2021-01-14,0.0
333469,7037057,1,41.15,2021-01-14,Labels,DOT Labels,2021-01-14,0.0


In [18]:
# Sum Quantity Order/Line Item Amount group by Customer No & Purchase date
summLabels = Labels.groupby(['Customer No', 'Posting Date', 'Days from 1st Purchase']).agg({'Quantity Ordered': 'sum', 'Line Item Amount': 'sum'}).reset_index()
summLabels.columns = ['Customer No', 'Posting Date', 'Days from 1st Purchase', 'Total Quantity Ordered', 'Total Line Item Amount']
# Deal with those who have only purchased once
remL= summLabels['Customer No'].value_counts().reset_index()
remL.columns = ['Customer No', 'Count']
summLabels = pd.merge(summLabels, remL, on='Customer No')
summLabels = summLabels[summLabels['Count'] != 1].reset_index(drop=True)
summLabels['Customer No'].value_counts()

0409806    1571
1458331    1235
0095457    1193
0202079    1044
0267242     923
           ... 
1293270       2
1293324       2
1293399       2
3028366       2
OWNY208       2
Name: Customer No, Length: 15801, dtype: int64

In [19]:
summLabels

,Customer No,Posting Date,Days from 1st Purchase,Total Quantity Ordered,Total Line Item Amount,Count
0,0000010,2016-04-27,0.0,2,128.20,5
1,0000010,2016-07-29,93.0,3,192.30,5
2,0000010,2016-10-18,174.0,6,166.80,5
3,0000010,2018-07-31,825.0,5,255.50,5
4,0000010,2018-08-01,826.0,8,298.40,5
...,...,...,...,...,...,...
142798,7052843,2022-03-23,2.0,9,112.59,2
142799,7053362,2022-03-31,0.0,2,68.22,2
142800,7053362,2022-04-04,4.0,4,284.96,2
142801,OWNY208,2016-05-27,0.0,1,15.50,2


In Label Department, there are 44,765 identical customers. There are only 15,801 customers have purchased more than once.

#### 2.1.1 Calculate boundary based on each customer's median & IQR

In [20]:
# Calculate the number of days between consecutive purchases
summLabels['Days from previous purchase'] = summLabels.groupby(['Customer No'])['Days from 1st Purchase'].diff().reset_index(drop = True)
summLabels = summLabels.dropna()
summLabels.shape

(127002, 7)

In [21]:
# Find median, IQR, lower bound, upper bound
medianL = summLabels.groupby('Customer No')['Days from previous purchase'].agg(['median', 'count']).reset_index()
iqrL = summLabels.groupby('Customer No')['Days from previous purchase'].agg(iqr).reset_index()
findingL = pd.merge(medianL, iqrL, on='Customer No')
findingL.columns = ['Customer No', 'Median', 'Count', 'IQR']
findingL['Lower Bound'] = findingL['Median'] - 3 * findingL['IQR'] / np.sqrt(findingL['Count'])
findingL['Upper Bound'] = findingL['Median'] + 3 * findingL['IQR'] / np.sqrt(findingL['Count'])
findingL

,Customer No,Median,Count,IQR,Lower Bound,Upper Bound
0,0000010,87.0,4,171.50,-170.250000,344.250000
1,0000145,93.5,10,205.25,-101.217247,288.217247
2,0000185,66.0,9,189.00,-123.000000,255.000000
3,0000258,20.0,1,0.00,20.000000,20.000000
4,0000323,43.0,5,47.00,-20.057117,106.057117
...,...,...,...,...,...,...
15796,7052591,5.0,1,0.00,5.000000,5.000000
15797,7052662,6.0,2,5.00,-4.606602,16.606602
15798,7052843,2.0,1,0.00,2.000000,2.000000
15799,7053362,4.0,1,0.00,4.000000,4.000000


In [22]:
getL = pd.merge(summLabels[['Customer No', 'Days from 1st Purchase', 'Days from previous purchase']], findingL[['Customer No', 'Lower Bound', 'Upper Bound']], on = 'Customer No')
getL

,Customer No,Days from 1st Purchase,Days from previous purchase,Lower Bound,Upper Bound
0,0000010,93.0,93.0,-170.250000,344.250000
1,0000010,174.0,81.0,-170.250000,344.250000
2,0000010,825.0,651.0,-170.250000,344.250000
3,0000010,826.0,1.0,-170.250000,344.250000
4,0000145,380.0,258.0,-101.217247,288.217247
...,...,...,...,...,...
126997,7052662,1.0,1.0,-4.606602,16.606602
126998,7052662,12.0,11.0,-4.606602,16.606602
126999,7052843,2.0,2.0,2.000000,2.000000
127000,7053362,4.0,4.0,4.000000,4.000000


In [ ]:
# # Find customers who have churned already based on last purchased date
# labeling1L = getL.groupby('Customer No').agg(lastPurchased = ('Days from Purchased to Now', np.min)).reset_index()
# labeling1L = pd.merge(getL[['Customer No', 'Lower Bound', 'Upper Bound']].drop_duplicates(), labeling1L, on='Customer No')
# labeling1L['Churn Definition 1'] = np.where((labeling1L['lastPurchased'] >= labeling1L['Lower Bound']) & (labeling1L['lastPurchased'] <= labeling1L['Upper Bound']), 0, 1)
# labeling1L['Churn Definition 1'].value_counts()

In [23]:
# Find customers who have churned already based on purchased history
countChurnL = getL.copy()
countChurnL['Churn Count'] = np.where((countChurnL['Days from previous purchase'] >= countChurnL['Lower Bound']) & (countChurnL['Days from previous purchase'] <= countChurnL['Upper Bound']), 0, 1)
resultLabels = countChurnL.groupby('Customer No')['Churn Count'].agg(['sum', 'count']).reset_index()
resultLabels.columns = ['Customer No', 'Churn Count', 'Purchase Count']
resultLabels['Churn Percent'] = round(resultLabels['Churn Count'] / resultLabels['Purchase Count'], 3)
resultLabels['Churn Definition'] = np.where(resultLabels['Churn Count'] > (resultLabels['Purchase Count']/3), 1, 0)
resultLabels['Churn Definition'].value_counts()

0    14320
1     1481
Name: Churn Definition, dtype: int64

In [24]:
resultLabels

,Customer No,Churn Count,Purchase Count,Churn Percent,Churn Definition
0,0000010,1,4,0.250,0
1,0000145,2,10,0.200,0
2,0000185,2,9,0.222,0
3,0000258,0,1,0.000,0
4,0000323,1,5,0.200,0
...,...,...,...,...,...
15796,7052591,0,1,0.000,0
15797,7052662,0,2,0.000,0
15798,7052843,0,1,0.000,0
15799,7053362,0,1,0.000,0


#### 2.1.2 Calculate bound based on each department's median & IQR

In [ ]:
# lowerBoundL = findingL['Lower Bound'].mean()
# upperBoundL = findingL['Upper Bound'].mean()

In [ ]:
# Find customers who have churned already based on last purchased date
# labeling3L = getL.groupby('Customer No').agg(lastPurchased = ('Days from Purchased to Now', np.min)).reset_index()
# labeling3L['Churn Definition 1'] = np.where((labeling3L['lastPurchased'] >= lowerBoundL) & (labeling3L['lastPurchased'] <= upperBoundL), 0, 1)
# labeling3L['Churn Definition 1'].value_counts()

In [ ]:
# Find customers who have churned already based on purchased history
# countChurn1L = getL.copy()
# countChurn1L['Churn Count'] = np.where(countChurn1L['Days from previous purchase'] > upperBoundL, 1, 0)
# countChurn1L
# labeling4L = pd.DataFrame(countChurn1L.groupby(['Customer No']).agg({'Churn Count': 'sum'})).reset_index()
# labeling4L['Churn Definition 2'] = np.where(labeling4L['Churn Count'] > 0, 1, 0)
# labeling4L['Churn Definition 2'].value_counts()

#### 2.1.3 Calculate bound based on all differences in Labels Department

In [ ]:
# q1L = previousL['Days from previous purchase'].quantile(.25)
# q3L = previousL['Days from previous purchase'].quantile(.75)
# IQRL = q3L - q1L
# MedianL = previousL['Days from previous purchase'].median()
# nL = previousL.shape[0]
# lowerL = MedianL - 4.5 * IQRL 
# upperL = MedianL + 4.5 * IQRL

In [ ]:
# Find customers who have churned already based on last purchased date
# labeling5L = getL.groupby('Customer No').agg(lastPurchased = ('Days from Purchased to Now', np.min)).reset_index()
# labeling5L['Churn Definition 1'] = np.where((labeling5L['lastPurchased'] >= lowerL) & (labeling5L['lastPurchased'] <= upperL), 0, 1)
# labeling5L
# labeling5L['Churn Definition 1'].value_counts()

In [ ]:
# Find customers who have churned already based on purchased history
# countChurn2L = getL.copy()
# countChurn2L['Churn Count'] = np.where((countChurn2L['Days from previous purchase'] >= lowerL) & (countChurn2L['Days from previous purchase'] <= upperL), 0, 1)
# countChurn2L
# labeling6L = pd.DataFrame(countChurn2L.groupby(['Customer No']).agg({'Churn Count': 'sum'})).reset_index()
# labeling6L['Churn Definition 2'] = np.where(labeling6L['Churn Count'] > 0, 1, 0)
# labeling6L['Churn Definition 2'].value_counts()

## 3. Group by Family's Industry

In [ ]:
# # Seperate into different Department
# industryName = myData['Family_Industry_Description_New__c'].unique()
# industryName

In [ ]:
# industry_sales.Family_Industry_Description_New__c.value_counts()

### 3.1 Example - look into Chemical Industry

In [ ]:
# # Seperate into different Industry
# Chemical = myData[myData['Family_Industry_Description_New__c'] == 'Chemical'].reset_index(drop = True)
# Chemical.shape

In [ ]:
# Chemical

In [ ]:
# Chemical['Customer No'].value_counts()

In [ ]:
# # Sum Quantity Order/Line Item Amount group by Customer No & Purchase date
# summChemical = Chemical.groupby(['Customer No', 'Posting Date', 'Days from 1st Purchase', 'Days from Purchased to Now']).agg({'Quantity Ordered': 'sum', 'Line Item Amount': 'sum'}).reset_index()
# summChemical.columns = ['Customer No', 'Posting Date', 'Days from 1st Purchase', 'Days from Purchased to Now', 'Total Quantity Ordered', 'Total Line Item Amount']
# # Deal with those who have only purchased once
# remC= summChemical['Customer No'].value_counts().reset_index()
# remC.columns = ['Customer No', 'Count']
# summChemical = pd.merge(summChemical, remC, on='Customer No')
# summChemical = summChemical[summChemical['Count'] != 1].reset_index(drop=True)
# summChemical['Customer No'].value_counts()

#### 3.1.1 Calculate bound based on each customer's median & IQR

In [ ]:
# # Calculate the number of days between consecutive purchases
# summChemical['Days from previous purchase'] = summChemical.groupby(['Customer No'])['Days from 1st Purchase'].diff().reset_index(drop = True)
# summChemical = summChemical.dropna()
# summChemical.shape

In [ ]:
# # Find median, IQR, lower bound, upper bound
# medianC = summChemical.groupby('Customer No')['Days from previous purchase'].agg(['median', 'count']).reset_index()
# iqrC = summChemical.groupby('Customer No')['Days from previous purchase'].agg(iqr).reset_index()
# findingC = pd.merge(medianC, iqrC, on='Customer No')
# findingC.columns = ['Customer No', 'Median', 'Count', 'IQR']
# findingC['Lower Bound'] = findingC['Median'] - 3 * findingC['IQR'] / np.sqrt(findingC['Count'])
# findingC['Upper Bound'] = findingC['Median'] + 3 * findingC['IQR'] / np.sqrt(findingC['Count'])
# findingC

In [ ]:
# getC = pd.merge(summChemical[['Customer No', 'Days from 1st Purchase', 'Days from Purchased to Now', 'Days from previous purchase']], findingC[['Customer No', 'Lower Bound', 'Upper Bound']], on = 'Customer No')
# getC

In [ ]:
# Find customers who have churned already based on last purchased date
# labeling1C = getC.groupby('Customer No').agg(lastPurchased = ('Days from Purchased to Now', np.min)).reset_index()
# labeling1C = pd.merge(getC[['Customer No', 'Lower Bound', 'Upper Bound']].drop_duplicates(), labeling1C, on='Customer No')
# labeling1C['Churn Definition 1'] = np.where((labeling1C['lastPurchased'] >= labeling1C['Lower Bound']) & (labeling1C['lastPurchased'] <= labeling1C['Upper Bound']), 0, 1)
# labeling1C['Churn Definition 1'].value_counts()

In [ ]:
# # Find customers who have churned already based on purchased history
# countChurnC = getC.copy()
# countChurnC['Churn Count'] = np.where((countChurnC['Days from previous purchase'] >= countChurnC['Lower Bound']) & (countChurnC['Days from previous purchase'] <= countChurnC['Upper Bound']), 0, 1)
# resultChemical = countChurnC.groupby('Customer No')['Churn Count'].agg(['sum', 'count']).reset_index()
# resultChemical.columns = ['Customer No', 'Churn Count', 'Purchase Count']
# resultChemical['Churn Percent'] = round(resultChemical['Churn Count'] / resultChemical['Purchase Count'], 3)
# resultChemical['Churn Definition'] = np.where(resultChemical['Churn Count'] > (resultChemical['Purchase Count']/3), 1, 0)
# resultChemical['Churn Definition'].value_counts()

In [ ]:
# resultChemical

#### 3.1.2 Calculate bound based on each industry's median & IQR

In [ ]:
# lowerBoundC = findingC['Lower Bound'].mean()
# upperBoundC = findingC['Upper Bound'].mean()

In [ ]:
# # Find customers who have churned already based on last purchased date
# labeling3C = getC.groupby('Customer No').agg(lastPurchased = ('Days from Purchased to Now', np.min)).reset_index()
# labeling3C['Churn Definition 1'] = np.where((labeling3C['lastPurchased'] >= lowerBoundC) & (labeling3C['lastPurchased'] <= upperBoundC), 0, 1)
# labeling3C['Churn Definition 1'].value_counts()

In [ ]:
# Find customers who have churned already based on purchased history
# countChurn1C = getC.copy()
# countChurn1C['Churn Count'] = np.where((countChurn1C['Days from previous purchase'] >= lowerBoundC) & (countChurn1C['Days from previous purchase'] <= upperBoundC), 0, 1)
# labeling4C = pd.DataFrame(countChurn1C.groupby(['Customer No']).agg({'Churn Count': 'sum'})).reset_index()
# labeling4C['Churn Definition 2'] = np.where(labeling4C['Churn Count'] > 0, 1, 0)
# labeling4C['Churn Definition 2'].value_counts()

#### 2.1.3 Calculate bound based on all differences in Chemical Industry

In [ ]:
# q1C = previousC['Days from previous purchase'].quantile(.25)
# q3C = previousC['Days from previous purchase'].quantile(.75)
# IQRC = q3C - q1C
# MedianC = previousC['Days from previous purchase'].median()
# nC = previousC.shape[0]
# lowerC = MedianC - 4.5 * IQRC
# upperC = MedianC + 4.5 * IQRC

In [ ]:
# # Find customers who have churned already based on last purchased date
# labeling5C = getC.groupby('Customer No').agg(lastPurchased = ('Days from Purchased to Now', np.min)).reset_index()
# labeling5C['Churn Definition 1'] = np.where((labeling5C['lastPurchased'] >= lowerC) & (labeling5C['lastPurchased'] <= upperC), 0, 1)
# labeling5C
# labeling5C['Churn Definition 1'].value_counts()

In [ ]:
# Find customers who have churned already based on purchased history
# countChurn2C = getC.copy()
# countChurn2C['Churn Count'] = np.where((countChurn2C['Days from previous purchase'] >= lowerC) & (countChurn2C['Days from previous purchase'] <= upperC), 0, 1)
# labeling6C = pd.DataFrame(countChurn2C.groupby(['Customer No']).agg({'Churn Count': 'sum'})).reset_index()
# labeling6C['Churn Definition 2'] = np.where(labeling6C['Churn Count'] > (findingC['Count']/2), 1, 0)
# # labeling6C['Churn Definition 2'] = np.where(labeling6C['Churn Count'] > 0, 1, 0)
# labeling6C['Churn Definition 2'].value_counts()

## 4. Combine the result

### 4.1 Function to define churn

In [25]:
def defineChurn(df):
    # Sum Quantity Order/Line Item Amount group by Customer No & Purchase date
    summ = df.groupby(['Customer No', 'Posting Date', 'Days from 1st Purchase']).agg({'Quantity Ordered': 'sum', 'Line Item Amount': 'sum'}).reset_index()
    summ.columns = ['Customer No', 'Posting Date', 'Days from 1st Purchase', 'Total Quantity Ordered', 'Total Line Item Amount']
    
    # Deal with those who have only purchased once
    rem= summ['Customer No'].value_counts().reset_index()
    rem.columns = ['Customer No', 'Count']
    summ = pd.merge(summ, rem, on='Customer No')
    one = summ[summ['Count'] == 1].reset_index(drop=True)
    summ = summ[summ['Count'] != 1].reset_index(drop=True)
    result = pd.DataFrame()
    
    if summ.empty == False:
        # Calculate the number of days between consecutive purchases
        summ['Days from previous purchase'] = summ.groupby(['Customer No'])['Days from 1st Purchase'].diff().reset_index(drop = True)
        summ = summ.dropna()
        
        # Find median, iqr, lower/upper bound
        Median = summ.groupby('Customer No')['Days from previous purchase'].agg(['median', 'count']).reset_index()
        IQR = summ.groupby('Customer No')['Days from previous purchase'].agg(iqr).reset_index()
        finding = pd.merge(Median, IQR, on='Customer No')
        finding.columns = ['Customer No', 'Median', 'Count', 'IQR']
        finding['Lower Bound'] = finding['Median'] - 3 * finding['IQR'] / np.sqrt(finding['Count'])
        finding['Upper Bound'] = finding['Median'] + 3 * finding['IQR'] / np.sqrt(finding['Count'])
        get = pd.merge(summ[['Customer No', 'Days from 1st Purchase', 'Days from previous purchase']], finding[['Customer No', 'Lower Bound', 'Upper Bound']], on = 'Customer No')
        
        # Calculate Churn times
        countChurn = get.copy()
        countChurn['Churn Count'] = np.where((countChurn['Days from previous purchase'] >= countChurn['Lower Bound']) & (countChurn['Days from previous purchase'] <= countChurn['Upper Bound']), 0, 1)
        result = countChurn.groupby('Customer No')['Churn Count'].agg(['sum', 'count']).reset_index()
        result.columns = ['Customer No', 'Churn Count', 'Purchase Count']
        result['Churn Percent'] = round(result['Churn Count'] / result['Purchase Count'], 3)
        result['Churn'] = np.where(result['Churn Count'] > (result['Purchase Count']/3), 1, 0)
        return one, result[['Customer No', 'Churn', 'Churn Percent']]
    return one, result

### 4.2 Department

In [26]:
moreDepartment = pd.DataFrame()
oneDepartment = pd.DataFrame()
for name in departmentName:
    df = myData[myData['Department Dim Description'] == name].reset_index(drop=True)
    one, r = defineChurn(df)
    oneDepartment = pd.concat([oneDepartment, one])
    if r.empty == False:
        r['Department'] = name
        moreDepartment = pd.concat([moreDepartment, r])

In [27]:
# In either department, customer has purchased only once
pd.DataFrame(oneDepartment['Customer No'].unique())

,0
0,0001697
1,0001791
2,0005573
3,0009901
4,0011647
...,...
75757,2500333
75758,2500334
75759,2500335
75760,2500338


In [28]:
moreDepartment

,Customer No,Churn,Churn Percent,Department
0,0000323,0,0.000,Software
1,0000678,0,0.333,Software
2,0000919,0,0.286,Software
3,0000969,0,0.000,Software
4,0001232,0,0.333,Software
...,...,...,...,...
170,2500336,0,0.000,LSI- Consulting
171,2500337,0,0.000,LSI- Consulting
172,2500342,0,0.000,LSI- Consulting
0,2500185,0,0.250,LSI- E-Learning


In [29]:
# Groupby each customer to see the overall performance in all Department
finalDepartment = moreDepartment.groupby('Customer No')['Churn'].agg(['count', 'sum']).reset_index()

In [30]:
# In either department, customer has purchased more than once
pd.DataFrame(finalDepartment['Customer No'].unique())

,0
0,.
1,0000010
2,0000133
3,0000145
4,0000185
...,...
35202,7053262
35203,7053362
35204,7053400
35205,7053526


Among 95,540 customers, there are 35,207 identical customers have purchased more than once while 75,762 customers have purchased only once. Some customers purchased only once in one department and purchased more than one in other departments.

In [31]:
finalDepartment.columns = ['Customer No', '# of Departments that customer purchased', '# of Departments that customer churned']
finalDepartment['Churn_Department'] = np.where(finalDepartment['# of Departments that customer churned'] > (finalDepartment['# of Departments that customer purchased']/3), 1, 0)
finalDepartment['Churn Percent_Department'] = round(finalDepartment['# of Departments that customer churned'] / finalDepartment['# of Departments that customer purchased'], 3)
finalDepartment

,Customer No,# of Departments that customer purchased,# of Departments that customer churned,Churn_Department,Churn Percent_Department
0,.,2,0,0,0.000
1,0000010,3,0,0,0.000
2,0000133,2,0,0,0.000
3,0000145,7,1,0,0.143
4,0000185,7,2,0,0.286
...,...,...,...,...,...
35202,7053262,1,0,0,0.000
35203,7053362,2,0,0,0.000
35204,7053400,1,0,0,0.000
35205,7053526,1,0,0,0.000


In [32]:
finalDepartment['Churn_Department'].value_counts()

0    32938
1     2269
Name: Churn_Department, dtype: int64

In [33]:
# Within-department churn rate
2269/(32938+2269)

0.06444741102621637

### 4.3 Product Line

In [34]:
myProduct = myData.copy()
# Replace Nan in Product Group Description with Department Dim Description
myProduct.loc[myProduct['Department Dim Description'] == 'LSI- Consulting', 'Product Group Description'] = 'LSI- Consulting'
myProduct.loc[(myProduct['Department Dim Description'] == 'Software') & (myProduct['Product Group Description'].isna()), 'Product Group Description'] = 'Software'
myProduct.loc[(myProduct['Department Dim Description'] == 'Contra Dimension') & (myProduct['Product Group Description'].isna()), 'Product Group Description'] = 'Contra Dimension'

In [35]:
myProduct = myProduct[myProduct['Product Group Description'].notnull()]

In [36]:
myProduct.shape

(1472243, 8)

In [37]:
myProduct.head()

,Customer No,Quantity Ordered,Line Item Amount,Posting Date,Department Dim Description,Product Group Description,First Purchase Date,Days from 1st Purchase
0,0000919,1,799.0,2018-04-01,Software,Software,2016-01-11,811.0
1,0000919,1,0.0,2018-04-01,Software,Software,2016-01-11,811.0
2,0000919,1,0.0,2018-04-01,Software,Software,2016-01-11,811.0
3,0000919,1,0.0,2018-04-01,Software,Software,2016-01-11,811.0
4,0000919,1,799.0,2018-04-01,Software,Software,2016-01-11,811.0


In [38]:
# Seperate into different Product Line
productName = myProduct['Product Group Description'].unique()
productName

array(['Software', 'Corrugated Shippers', 'Forms', 'DOT Labels',
       'Contra Dimension', 'International Air Transport Association',
       'Pre-Printed Placards', 'Air Labels', 'International Labels',
       'Hazcom Labels', 'Worded Placards',
       'Personalized DOT Labels and Markings', 'Warehouse Labels',
       'Early 49 CFR', 'Regulatory Markings', 'Seals',
       'Code of Federal Regulations', 'Imprinted One Sided Placards',
       'Placard Accessories', 'Blank Placards', 'ERG',
       'MasterRegs 49 CFR', 'Variation Packaging',
       'International Maritime Organization', 'Masterform New',
       'Waste Labels', 'Exemption Packaging 173.3', 'Reference Books',
       'Personalized Warehouse Labels', 'Shipping Supplies',
       'Custom Labels', 'Personalized Waste Lables',
       'Orange Panels & Markings', 'Poly Drums and Pails',
       'Chemtrec Markings', 'Paint Can Shippers',
       'Personalized Regulatory Markings', 'Metal Packaging',
       'Foam Shippers', 'GHS Traini

In [39]:
moreProduct = pd.DataFrame()
oneProduct = pd.DataFrame()
for name in productName:
    df = myProduct[myProduct['Product Group Description'] == name].reset_index(drop=True)
    one, r = defineChurn(df)
    oneProduct = pd.concat([oneProduct, one])
    if r.empty == False:
        r['Product'] = name
        moreProduct = pd.concat([moreProduct, r])

In [40]:
# In either product line, customer has purchased only once
pd.DataFrame(oneProduct['Customer No'].unique())

,0
0,0000323
1,0008240
2,0013313
3,0017066
4,0025872
...,...
83682,2500334
83683,2500335
83684,2500338
83685,2500340


In [41]:
moreProduct

,Customer No,Churn,Churn Percent,Product
0,0000919,0,0.286,Software
1,0001467,0,0.333,Software
2,0002083,0,0.000,Software
3,0004448,0,0.286,Software
4,0004486,0,0.200,Software
...,...,...,...,...
169,2500323,0,0.000,LSI- Consulting
170,2500336,0,0.000,LSI- Consulting
171,2500337,0,0.000,LSI- Consulting
172,2500342,0,0.000,LSI- Consulting


In [42]:
moreProduct['Product'].value_counts()

Contra Dimension           30752
DOT Labels                  5171
Pre-Printed Placards        4106
Worded Placards             3351
Air Labels                  3187
                           ...  
Inactive                       9
Amazon Packaging               6
Amazon Warehouse Labels        4
GM Packaging                   1
GM Labor                       1
Name: Product, Length: 77, dtype: int64

In [43]:
# Groupby each customer to see the overall performance in all Product Line
finalProduct = moreProduct.groupby('Customer No')['Churn'].agg(['count', 'sum']).reset_index()

In [44]:
# In either product line, customer has purchased more than once
pd.DataFrame(finalProduct['Customer No'].unique())

,0
0,.
1,0000010
2,0000133
3,0000145
4,0000185
...,...
34646,7053262
34647,7053362
34648,7053400
34649,7053526


Among 95,540 customers, there are 34,651 identical customers have purchased more than once while 83,687 customers have purchased only once. Some customers purchased only once in one department and purchased more than one in other departments.

In [45]:
finalProduct.columns = ['Customer No', '# of Product Line that customer purchased', '# of Product Line that customer churned']
finalProduct['Churn_Product'] = np.where(finalProduct['# of Product Line that customer churned'] > (finalProduct['# of Product Line that customer purchased']/3), 1, 0)
finalProduct['Churn Percent_Product'] = round(finalProduct['# of Product Line that customer churned'] / finalProduct['# of Product Line that customer purchased'], 3)
finalProduct

,Customer No,# of Product Line that customer purchased,# of Product Line that customer churned,Churn_Product,Churn Percent_Product
0,.,2,0,0,0.000
1,0000010,4,0,0,0.000
2,0000133,1,0,0,0.000
3,0000145,19,3,0,0.158
4,0000185,12,2,0,0.167
...,...,...,...,...,...
34646,7053262,1,0,0,0.000
34647,7053362,1,0,0,0.000
34648,7053400,1,0,0,0.000
34649,7053526,1,0,0,0.000


In [46]:
finalProduct['Churn_Product'].value_counts()

0    33294
1     1357
Name: Churn_Product, dtype: int64

In [47]:
# Within-industry churn rate
1357/(33294+1357)

0.03916192894865949

### 4.4 Industry

In [ ]:
# finalIndustry = pd.DataFrame()
# oneIndustry = pd.DataFrame()
# for name in industryName:
#     df = myData[myData['Family_Industry_Description_New__c'] == name].reset_index(drop=True)
#     one, r = defineChurn(df)
#     oneIndustry = pd.concat([oneIndustry, one])
#     if r.empty == False:
#         r['Industry'] = name
#         finalIndustry = pd.concat([finalIndustry, r])

In [ ]:
# oneIndustry['Customer No']

In [ ]:
# finalIndustry.columns = ['Customer No', 'Churn_Industry', 'Churn Percent_Industry', 'Industry']
# finalIndustry

In [ ]:
# finalIndustry['Churn_Industry'].value_counts()

### 4.5 Department & Industry

In [ ]:
# combine = pd.merge(finalIndustry[['Customer No', 'Churn_Industry', 'Churn Percent_Industry', 'Industry']], 
#                  finalDepartment[['Customer No', 'Churn_Department', 'Churn Percent_Department']], 
#                  on = 'Customer No', how = 'outer')

In [ ]:
# pd.crosstab(combine['Churn_Industry'], combine['Churn_Department'], margins=True, margins_name="Total")

In [ ]:
# # Churn Define for those who have purchased >1 in total, but only once in either department
# part = combine[combine.isna().any(axis=1)][['Customer No', 'Churn_Industry']].reset_index(drop=True)
# part.columns = ['Customer No', 'Churn']
# part['Churn'] = 'Not Churn'

In [ ]:
# notChurn = combine[(combine['Churn_Industry'] == 0) & (combine['Churn_Department'] == 0)][['Customer No']].reset_index(drop=True)
# notChurn['Churn'] = 'Not Churn'

In [ ]:
# wayChurn = combine[((combine['Churn_Industry'] == 0) & (combine['Churn_Department'] == 1)) | ((combine['Churn_Industry'] == 1) & (combine['Churn_Department'] == 0))][['Customer No']].reset_index(drop=True)
# wayChurn['Churn'] = 'On the way of Churning'

In [ ]:
# Churn = combine[(combine['Churn_Industry'] == 1) & (combine['Churn_Department'] == 1)][['Customer No']].reset_index(drop=True)
# Churn['Churn'] = 'Churn'

In [ ]:
# final = pd.concat([part, notChurn, wayChurn, Churn])

In [ ]:
# final['Churn'].value_counts()